<a href="https://colab.research.google.com/github/santoryudeadman/whisper/blob/main/bakllava.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip
!./ngrok authtoken 2XMswsT2z3mRwXoEkALduNhNDoA_7tfvtmZ56wGnt7qebcjHd

--2023-10-28 04:36:58--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 54.161.241.46, 54.237.133.81, 18.205.222.128, ...
Connecting to bin.equinox.io (bin.equinox.io)|54.161.241.46|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13921656 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]  13.28M  21.7MB/s    in 0.6s    

2023-10-28 04:36:59 (21.7 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [13921656/13921656]

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   
Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [ ]:
!sudo apt update -y && sudo apt install aria2 build-essential -y
!aria2c -x4 -o ggml-model-q5_k.gguf https://huggingface.co/mys/ggml_bakllava-1/resolve/main/ggml-model-q5_k.gguf
!aria2c -x4 -o mmproj-model-f16.gguf https://huggingface.co/mys/ggml_bakllava-1/resolve/main/mmproj-model-f16.gguf


Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:6 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ Packages [44.8 kB]
Get:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [109 kB]
Hit:8 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Get:9 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,274 kB]
Hit:10 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:11 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [1,400 kB]
Get:12 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [1,131 kB]
Hit:13 https://ppa.launchpadcontent.net/gra

In [ ]:
!git clone https://github.com/ggerganov/llama.cpp && cd llama.cpp && make LLAMA_CUBLAS=1

Cloning into 'llama.cpp'...
remote: Enumerating objects: 11482, done.
remote: Counting objects: 100% (2835/2835), done.
remote: Compressing objects: 100% (184/184), done.
remote: Total 11482 (delta 2737), reused 2674 (delta 2651), pack-reused 8647
Receiving objects: 100% (11482/11482), 13.54 MiB | 19.12 MiB/s, done.
Resolving deltas: 100% (8003/8003), done.
I llama.cpp build info: 
I UNAME_S:   Linux
I UNAME_P:   x86_64
I UNAME_M:   x86_64
I CFLAGS:    -I. -Icommon -D_XOPEN_SOURCE=600 -D_GNU_SOURCE -DNDEBUG -DGGML_USE_K_QUANTS -DGGML_USE_CUBLAS -I/usr/local/cuda/include -I/opt/cuda/include -I/targets/x86_64-linux/include  -std=c11   -fPIC -O3 -Wall -Wextra -Wpedantic -Wcast-qual -Wno-unused-function -Wshadow -Wstrict-prototypes -Wpointer-arith -Wmissing-prototypes -Werror=implicit-int -Werror=implicit-function-declaration -Wdouble-promotion -pthread -march=native -mtune=native 
I CXXFLAGS:  -I. -Icommon -D_XOPEN_SOURCE=600 -D_GNU_SOURCE -DNDEBUG -DGGML_USE_K_QUANTS -DGGML_USE_CUBLAS -I

In [ ]:
config = """
authtoken: 2XMswsT2z3mRwXoEkALduNhNDoA_7tfvtmZ56wGnt7qebcjHd
region: us
tunnels:
  first_tunnel:
    proto: http
    addr: 127.0.0.1:1000
    hostname: big-poodle-upward.ngrok-free.app
  second_tunnel:
    proto: http
    addr: 127.0.0.1:80
"""

with open('/root/.ngrok2/ngrok.yml', 'w') as f:
    f.write(config)


In [ ]:
!pip install flask flask-ngrok
from flask import Flask
from flask_ngrok import run_with_ngrok
import subprocess
import threading
import time
from IPython.display import IFrame

app = Flask(__name__)
run_with_ngrok(app)   #starts ngrok when the app is run

@app.route("/")
def home():
    return "<h1>Running Flask on Google Colab!</h1>"

# Function to start the server
def start_server():
    subprocess.Popen(["./server", "--port", "1000", "-ngl", "45", "-m", "../ggml-model-q5_k.gguf", "--mmproj", "../mmproj-model-f16.gguf", "-t", "2"], cwd="/content/llama.cpp/")

# Start the server in a separate thread
threading.Thread(target=start_server).start()

# Give the server time to start
time.sleep(2)

# Terminate any existing ngrok processes
get_ipython().system_raw('pkill -f ngrok')

# Start ngrok with the start command and the --all option:
get_ipython().system_raw('./ngrok start --all -config=/content/ngrok.yml > ngrok.log &')

# Display the server's content in an iframe
IFrame("http://localhost:1000", width=1024, height=768)


In [ ]:
get_ipython().system_raw('./ngrok http 1000 -log=stdout > ngrok.log &')


In [ ]:
# don't worry above is a dummy api key that's been reset , that's what it looks like CHANGE IT WITH YOUR OWN
!cat ngrok.log      #look below for the http and https links i.e.  https://d06a-34-87-173-22.ngrok-free.app
#Then go on your home computer and open the tunnel to the webserver on this lab and use the model :)

t=2023-10-28T04:40:04+0000 lvl=info msg="no configuration paths supplied"
t=2023-10-28T04:40:04+0000 lvl=info msg="using configuration at default config path" path=/root/.ngrok2/ngrok.yml
t=2023-10-28T04:40:04+0000 lvl=info msg="open config file" path=/root/.ngrok2/ngrok.yml err=nil
t=2023-10-28T04:40:04+0000 lvl=info msg="starting web service" obj=web addr=127.0.0.1:4040
t=2023-10-28T04:40:04+0000 lvl=info msg="tunnel session started" obj=tunnels.session
t=2023-10-28T04:40:04+0000 lvl=info msg="client session established" obj=csess id=c0c6ab1df320
t=2023-10-28T04:40:05+0000 lvl=info msg="started tunnel" obj=tunnels name="command_line (http)" addr=http://localhost:1000 url=http://d476-34-125-137-51.ngrok-free.app
t=2023-10-28T04:40:05+0000 lvl=info msg="started tunnel" obj=tunnels name=command_line addr=http://localhost:1000 url=https://d476-34-125-137-51.ngrok-free.app
